### 0. Kết nối các thư viện, ggsheet, khai báo hàm
***

In [1]:
# thư viện data, tính toán
import pandas as pd
import numpy as np
import re
import math

# query sql
import pypyodbc as py

# các thư viện làm việc với ggsheet
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread_dataframe as gd #update data vào sheet
from oauth2client.service_account import ServiceAccountCredentials # Đọc dữ liệu từ googlesheets
from google.oauth2 import service_account
from df2gspread import df2gspread as d2g
import json
import pygsheets # cấp quyền edit
import pymssql

# thư viện ngày giờ
import time
import datetime as dt
from datetime import datetime, timedelta
from math import radians, cos, sin, sqrt, atan2
from datetime import date
from time import strftime

import seaborn as sns
%matplotlib inline

In [2]:
db = py.connect("Driver={ODBC Driver 17 for SQL Server};"
               "Server=103.69.193.246;"
                "Database=dwh;"
                "uid=dwh_qtrr;pwd=ztc@247we4")
print('Connected to SQL server: DWH')

Connected to SQL server: DWH


In [3]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\Admin\Desktop\Code\jupyter-GGsheet\service_account.json', scope)
client = gspread.authorize(credentials)

In [4]:
wb_id = '1ccn3vGeiYGRr08z8z-sNkyVjL7gGF9jwKz9ibMyovX0'
sheet = client.open_by_key(wb_id)

In [5]:
#Function chuyển sheet thành Data Frame
def to_dataframe(wb_name,sheet_name,data_row,col):
    a=wb_name.worksheet(sheet_name).get_all_values()
    return pd.DataFrame(data=a[data_row:],columns=a[col])

#Funtion update data vào sheet wb_name:tên wb,target_sheet: tên sheet cần update; data:dữ liệu muốn update
def update_data(wb_name,target_sheet,data):
    existing =to_dataframe(wb_name,target_sheet,1,0)
    updated = existing.append(data)
    gd.set_with_dataframe(wb_name.worksheet(target_sheet),updated)

# Function ghi đè dữ liệu lên gg sheet
def write_data(wb_name,ws_name,ws_range,data):
    wb_name.values_clear(ws_range) 
    sheet=wb_name.worksheet(ws_name)
    set_with_dataframe(sheet,data)
    print('Đã ghi dữ liệu lên sheet '+ ws_name)

In [6]:
df = pd.read_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\data.1006.xlsx',sheet_name='Sheet2')

In [7]:
df.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data'],
      dtype='object')

### 1. Lấy tọa độ, thời gian log chip
***

#### i. Lấy data raw tọa độ, tgian

In [8]:
sub_df = df.loc[:,['event_data']]

In [9]:
s = sub_df['event_data'].str.split('{').apply(pd.Series,1).stack()
s.name = 'split_data'
s. index = s.index.droplevel(-1)

In [10]:
print(s.iloc[1])

'time': '190521095514', 'event_type': '4', 'lat': '10.2302275', 'dc': '0', 'event_data': 'ACC_SLEEP', 'lng': '105.98792666666667', 'timestamp': 1621418114}, 


In [11]:
sdf = df.join(s)

In [12]:
sdf.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data', 'split_data'],
      dtype='object')

##### Note: Để ý rules các tag dữ liệu long-lat. event ... để điều chỉnh extract bằng regex ra đúng

In [54]:
def get_longitude(cell):
    x = re.search('lng\': \'(.+?)\',', cell)
    if x is not None:
        return(x.group(1))

In [55]:
def get_latitude(cell):
    x = re.search('lat\': \'(.+?)\',',cell)
    if x is not None:
        return(x.group(1))

In [56]:
def get_time(cell):
    x = re.search('timestamp\':(.+?)}',cell)
    if x is not None:
        try:
            log_time = int(x.group(1))
            return (datetime.utcfromtimestamp(log_time)+timedelta(hours=7)).strftime('%Y-%m-%d %H:%M:%S')
        except:
            return None

In [16]:
def get_event(cell):
    x = re.search('event_data\': \'(.+?)\',',cell)
    if x is not None:
        return(x.group(1))  

In [57]:
sdf['longitude'] = sdf['split_data'].apply(lambda row:get_longitude(row))

In [58]:
sdf['latitude'] = sdf['split_data'].apply(lambda row:get_latitude(row))

In [59]:
sdf['get_time'] = sdf['split_data'].apply(lambda row:get_time(row))

In [60]:
sdf['event_log'] = sdf['split_data'].apply(lambda row:get_event(row))

In [61]:
sdf.columns

Index(['Imei', 'contract', 'open_time', 'last_event', 'event', 'no_signal_48h',
       'no_signal_days', 'Nghi Tháo nguồn', '12VDC', 'FW', 'Nghi Acquy kém',
       'DC_LOSS/PLUG gần nhất', 'DC_LOSS/PLUG trước đó',
       'Tọa độ DC_LOSS/PLUG gần nhất', 'Tọa độ DC_LOSS/PLUG trước đó',
       'event_data', 'split_data', 'longitude', 'latitude', 'get_time',
       'event_log'],
      dtype='object')

In [62]:
sdf

,Imei,contract,open_time,last_event,event,no_signal_48h,no_signal_days,Nghi Tháo nguồn,12VDC,FW,...,DC_LOSS/PLUG gần nhất,DC_LOSS/PLUG trước đó,Tọa độ DC_LOSS/PLUG gần nhất,Tọa độ DC_LOSS/PLUG trước đó,event_data,split_data,longitude,latitude,get_time,event_log
0,A006876,NaN,NaN,2021-05-19 16:55:24,ACC_SLEEP,0,0,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'time': '190521095514', 'event_type': '4', '...",[,None,None,None,None
0,A006876,NaN,NaN,2021-05-19 16:55:24,ACC_SLEEP,0,0,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'time': '190521095514', 'event_type': '4', '...","'time': '190521095514', 'event_type': '4', 'la...",105.98792666666667,10.2302275,2021-05-19 16:55:14,ACC_SLEEP
0,A006876,NaN,NaN,2021-05-19 16:55:24,ACC_SLEEP,0,0,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'time': '190521095514', 'event_type': '4', '...","'time': '190521094115', 'event_type': '3', 'la...",105.988015,10.2302115,2021-05-19 16:41:15,ACC_WAKE
0,A006876,NaN,NaN,2021-05-19 16:55:24,ACC_SLEEP,0,0,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'time': '190521095514', 'event_type': '4', '...","'time': '190521093555', 'event_type': '4', 'la...",105.988015,10.2302115,2021-05-19 16:35:55,ACC_SLEEP
0,A006876,NaN,NaN,2021-05-19 16:55:24,ACC_SLEEP,0,0,0,0,3.0.1604290516,...,2021-05-19 14:17:55:DC_LOSS,2021-05-19 14:14:15:DC_PLUG,"10.244295666666666,105.98059166666667","10.244295666666666,105.98059166666667","[{'time': '190521095514', 'event_type': '4', '...","'time': '190521092555', 'event_type': '3', 'la...",105.98776166666667,10.229982666666666,2021-05-19 16:25:55,ACC_WAKE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-10 01:53:59,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'time': '090621185353', 'event_type': '5', '...","'time': '040621080125', 'event_type': '4', 'la...",106.46953,9.614372166666667,2021-06-04 15:01:25,ACC_SLEEP
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-10 01:53:59,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'time': '090621185353', 'event_type': '5', '...","'time': '040621074237', 'event_type': '3', 'la...",106.43238,9.609984833333334,2021-06-04 14:42:37,ACC_WAKE
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-10 01:53:59,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'time': '090621185353', 'event_type': '5', '...","'time': '040621070937', 'event_type': '4', 'la...",106.43238,9.609984833333334,2021-06-04 14:09:37,ACC_SLEEP
9573,A000606,1747613,2021-04-22 16:12:14,2021-06-10 01:53:59,RTC_WAKE,0,0,0,0,3.0.0,...,NaN,NaN,NaN,NaN,"[{'time': '090621185353', 'event_type': '5', '...","'time': '040621062857', 'event_type': '3', 'la...",106.49231666666667,9.638759666666667,2021-06-04 13:28:57,ACC_WAKE


#### ii. Clean dữ liệu

In [63]:
coord_chip = sdf[['Imei','contract','open_time','event_log','longitude','latitude','get_time']]

In [64]:
coord_chip.columns

Index(['Imei', 'contract', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [65]:
coord_chip.reset_index(drop=True, inplace=True)

In [66]:
coord_chip.rename(columns = {'contract':'pawnid'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [67]:
data_chip = coord_chip.dropna(subset=['pawnid'])

In [68]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [69]:
data_chip['len_pid'] = data_chip['pawnid'].apply(len)

<ipython-input-69-4c92ad4d3e6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_chip['len_pid'] = data_chip['pawnid'].apply(len)


In [70]:
data_chip['len_pid'].unique()

array([ 7, 19, 17, 18], dtype=int64)

In [71]:
data_chip_clean = data_chip[data_chip['len_pid'] ==7]

In [72]:
data_chip_clean.drop(columns='len_pid',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [73]:
df2=data_chip_clean

In [74]:
df2 = df2.drop(df2[df2.get_time == None].index)

In [75]:
df2.reset_index(drop=True, inplace=True)

In [76]:
df2.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [77]:
df3 = df2.dropna(subset=['get_time','longitude','latitude'])

In [78]:
df3.reset_index(drop=True, inplace=True)

In [79]:
df3['pawnid'] = pd.to_numeric(df3['pawnid'])
df3['longitude'] = pd.to_numeric(df3['longitude'])
df3['latitude'] = pd.to_numeric(df3['latitude'])

<ipython-input-79-447850962fda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['pawnid'] = pd.to_numeric(df3['pawnid'])
<ipython-input-79-447850962fda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['longitude'] = pd.to_numeric(df3['longitude'])
<ipython-input-79-447850962fda>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [80]:
df3.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time'],
      dtype='object')

In [81]:
df3.isna().sum()

Imei         0
pawnid       0
open_time    0
event_log    0
longitude    0
latitude     0
get_time     0
dtype: int64

In [82]:
df3

,Imei,pawnid,open_time,event_log,longitude,latitude,get_time
0,A012966,1713035,2021-03-25 17:01:27,RTC_WAKE,105.334190,21.486577,2021-06-10 00:42:40
1,A012966,1713035,2021-03-25 17:01:27,RTC_WAKE,105.334190,21.486577,2021-06-09 22:44:47
2,A012966,1713035,2021-03-25 17:01:27,RTC_WAKE,105.334190,21.486577,2021-06-09 20:46:55
3,A012966,1713035,2021-03-25 17:01:27,RTC_WAKE,105.334190,21.486577,2021-06-09 18:49:05
4,A012966,1713035,2021-03-25 17:01:27,ACC_SLEEP,105.334190,21.486577,2021-06-09 16:48:45
...,...,...,...,...,...,...,...
716635,A000606,1747613,2021-04-22 16:12:14,ACC_SLEEP,106.469530,9.614372,2021-06-04 15:01:25
716636,A000606,1747613,2021-04-22 16:12:14,ACC_WAKE,106.432380,9.609985,2021-06-04 14:42:37
716637,A000606,1747613,2021-04-22 16:12:14,ACC_SLEEP,106.432380,9.609985,2021-06-04 14:09:37
716638,A000606,1747613,2021-04-22 16:12:14,ACC_WAKE,106.492317,9.638760,2021-06-04 13:28:57


### 2.  2a.Lấy tọa độ nhà khách hàng
***

In [42]:
#Lấy tọa độ của nhà KH
get_data ="""
    select shop_name,province,pawnid,
            customerID,codeno, fromdate, long_lat 
    from(
         select shop_name,province,row_number() over (partition by pawnid order by pawnid asc) row_id,
                pawnid,customerID,codeno, fromdate, long_lat 
         from pawn
         left join (select CUSTOMER_WID, long_lat 
                    from W_PAWN_CUSTOEMR_F
                        )cs on cs.CUSTOMER_WID = pawn.CustomerID

         left join W_STORE_SERIES_D sd on sd.OPEN_DT = pawn.shopcode
         where long_lat is not null
         )tab


    where row_id = 1
    """
data = pd.read_sql_query(get_data,db)


In [43]:
data_customer = data

In [44]:
new = data_customer["long_lat"].str.split(",", n=1, expand=True)

In [45]:
new.columns = ['latitude_cus','longitude_cus']

In [46]:
data_customer = pd.concat([data_customer,new], axis=1)

In [47]:
data_customer.columns

Index(['shop_name', 'province', 'pawnid', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus'],
      dtype='object')

In [48]:
def get_numeric(x):
    try:
        return pd.to_numeric(x)
    except:
        return None

In [49]:
data_customer['latitude_cus'] = data_customer['latitude_cus'].apply(get_numeric)
data_customer['longitude_cus'] = data_customer['longitude_cus'].apply(get_numeric)

In [50]:
data_customer2 = data_customer.dropna(subset=['latitude_cus'])

In [51]:
data_customer2 = data_customer2.dropna(subset=['longitude_cus'])

In [52]:
data_customer2.isna().sum()

shop_name        2024
province         2024
pawnid              0
customerid          0
codeno              0
fromdate            0
long_lat            0
latitude_cus        0
longitude_cus       0
dtype: int64

### 2. 2b TEST Lấy rule relocate
***

In [ ]:
df_re = df3[['longitude','latitude']].astype(str).apply(' '.join, axis=1)
df_re.name = 'longlat'

In [ ]:
df_relocate = df3.join(df_re)

In [ ]:
# tách riêng ngày - giờ 
split_time = df_relocate['get_time'].str.split(" ", n=1, expand=True)
split_time.columns = ['date_ym_log','time_log']
df_relocate = pd.concat([df_relocate,split_time], axis=1)

In [ ]:
df_relocate.reset_index(inplace = True)

In [ ]:
#df_relocate2 = df_relocate[df_relocate['row_id'] == 1]

In [ ]:
data_relocate = pd.merge(df_relocate, data_customer2,on='pawnid', how='left')

In [ ]:
data_relocate.columns

In [ ]:
data_relo2 = data_relocate.dropna(subset=['latitude_cus'])

In [ ]:
ad_data = data_relo2['time_log'].str.split(':', n=2, expand = True)
ad_data.columns = ['hour','minute','second']
data_relo2 = pd.concat([data_relo2,ad_data], axis = 1)

In [ ]:
def get_hour_point(row):
    if int(row) < 30:
        return 0
    elif int(row) >= 30:
        return 0.5

In [ ]:
data_relo2['get_time_hour'] = data_relo2['hour'].apply(int) + data_relo2['minute'].apply(lambda row: get_hour_point(row))

In [ ]:
data_relo2['date_hour_group'] = data_relo2['date_ym_log']
data_relo2 = data_relo2.sort_values(by=['pawnid','date_ym_log','time_log'])

In [ ]:
data_relo2.columns

In [ ]:
fn = lambda x: radians(x)

In [ ]:
data_relo2['long_rad'] = data_relo2['longitude'].apply(fn)
data_relo2['lat_rad'] = data_relo2['latitude'].apply(fn)

data_relo2['long_rad_cs'] = data_relo2['longitude_cus'].apply(fn)
data_relo2['lat_rad_cs'] = data_relo2['latitude_cus'].apply(fn)

In [ ]:
def get_distance(lat_cus,long_cus,lat_chip,long_chip):
    R = 6373.0
    dlon = long_chip - long_cus
    dlat = lat_chip - lat_cus

    a = sin(dlat / 2)**2 + cos(lat_cus) * cos(lat_chip) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# define a function for the new column
fn = lambda row:get_distance(row.lat_rad_cs,row.long_rad_cs,row.lat_rad,row.long_rad)

In [ ]:
# get column data with an index
col = data_relo2.apply(fn,axis=1)

In [ ]:
# assign values to column 'distance_km'
data_relo3 = data_relo2.assign(distance_km = col.values)

In [ ]:
data_relo4  = pd.DataFrame(data_relo3[['pawnid','Imei','shop_name','province','distance_km','get_time_hour','date_ym_log','longlat']])

In [ ]:
data_relo4['date-hour'] = data_relo4['date_ym_log'] + '-' + data_relo4['get_time_hour'].apply(str)

In [ ]:
# Chỉ lấy 1 dòng log-event theo ngày giờ
data_relo4 = data_relo4.sort_values(by=['pawnid', 'date-hour'], ascending = True)
data_relo4['row_id'] = data_relo4.groupby(['pawnid','date-hour']).cumcount()+1

In [ ]:
data_relo5 = data_relo4[data_relo4['row_id'] == 1]

In [ ]:
data_relo5.reset_index(drop=True, inplace=True)

In [ ]:
data_relo5

In [ ]:
data_relo5 = data_relo5.sort_values(by=['pawnid','date_ym_log','get_time_hour'], ascending=True)

In [ ]:
data_relo5.reset_index(drop = True,inplace=True)

In [ ]:
#Lấy các khoảng giao giữa ngày
a = data_relo5['date_ym_log'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
b = a.diff().apply(lambda x: x.days)
b.name = 'date_diff'

In [ ]:
#Đo khoảng thời gian giữa các lần log định vị
c = data_relo5.groupby('pawnid')['get_time_hour'].apply(lambda x: x.diff())
c. name = 'hour_diff'

In [ ]:
#So sánh long-lat giữa 2 thời điểm
d = data_relo5['longlat'] == data_relo5['longlat'].shift()
d.name = 'longlat_diff'

In [ ]:
data_relo5 = pd.concat([data_relo5,b,c,d], axis=1)

In [ ]:
def get_duration(row):
    if (row['date_diff'] == 0) and (row['longlat_diff'] == True) and (row['hour_diff'] <=2):
        return row['hour_diff']
    elif (row['date_diff']  == 1) and (row['longlat_diff'] == True) and ((row['hour_diff']+24) <= 2):
        return row['hour_diff']+24
    else:
        return 0

In [ ]:
f = data_relo5.apply(lambda row: get_duration(row), axis=1)
f.name = 'duration'

In [ ]:
data_relo5 = data_relo5.join(f)

In [ ]:
data_relo5.columns

In [ ]:
data_relo6 = data_relo5[['pawnid','Imei','longlat','date_ym_log','get_time_hour','duration','distance_km']]

In [ ]:
data_relo6['cum_dur'] = 0

In [ ]:
for i in range(1,len(data_relo6.index)):
    if data_relo6['longlat'][i] == data_relo6['longlat'][i-1]:
        data_relo6['cum_dur'][i] = data_relo6['cum_dur'][i-1] + data_relo6['duration'][i]
    else:
        data_relo6['cum_dur'][i] = 0  

In [ ]:
data_relo7 = data_relo6.groupby(['pawnid','Imei','distance_km','longlat','date_ym_log'])['cum_dur'].max()

In [ ]:
data_relo7 = pd.DataFrame(data_relo7)

In [ ]:
data_relo7

In [ ]:
data_relo7 = data_relo7.sort_values(by=['pawnid','date_ym_log','cum_dur'], ascending=False)
data_relo7['row_id'] = data_relo7.groupby(['pawnid','date_ym_log']).cumcount()+1

In [ ]:
data_relo7

In [ ]:
# Lấy top tọa độ theo từng ngày
data_relo8 = data_relo7[data_relo7['row_id'] <=5 ]

In [ ]:
# Lấy tọa độ xuất hiện nhiều nhất của tháng 
data_relo8 = data_relo8.sort_values(by=['pawnid','cum_dur'], ascending = False)
data_relo8['row_id'] = data_relo7.groupby(['pawnid']).cumcount()+1

In [ ]:
data_relo9.to_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\test3.xlsx')

### 3. Merge 2 bảng nhà KH - định vị chip
***

In [83]:
data_chip = pd.merge(df3, data_customer2,on='pawnid', how='left')

In [84]:
data_chip.columns


Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus'],
      dtype='object')

### 4. Tính khoảng cách từ vị trí
***

In [85]:
fn = lambda x: radians(x)

In [86]:
data_chip['long_rad'] = data_chip['longitude'].apply(fn)
data_chip['lat_rad'] = data_chip['latitude'].apply(fn)

data_chip['long_rad_cs'] = data_chip['longitude_cus'].apply(fn)
data_chip['lat_rad_cs'] = data_chip['latitude_cus'].apply(fn)

In [87]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus', 'long_rad', 'lat_rad',
       'long_rad_cs', 'lat_rad_cs'],
      dtype='object')

In [88]:
def get_distance(lat_cus,long_cus,lat_chip,long_chip):
    R = 6373.0
    dlon = long_chip - long_cus
    dlat = lat_chip - lat_cus

    a = sin(dlat / 2)**2 + cos(lat_cus) * cos(lat_chip) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [89]:
# define a function for the new column
fn = lambda row:get_distance(row.lat_rad_cs,row.long_rad_cs,row.lat_rad,row.long_rad)

In [90]:
# get column data with an index
col = data_chip.apply(fn,axis=1)

In [91]:
# assign values to column 'distance_km'
data_chip = data_chip.assign(distance_km = col.values)

### 5. Thống kê số lần xe đi vào bán kính
***

In [92]:
# tách riêng ngày - giờ 
split_time = data_chip['get_time'].str.split(" ", n=1, expand=True)
split_time.columns = ['date_ym_log','time_log']
data_chip = pd.concat([data_chip,split_time], axis=1)

##### Đặt điều kiện trong bán kính nhà khách hàng

In [93]:
def condition(row):
    if row['distance_km'] > 2:
        return 0
    else:
        return 1

In [94]:
data_chip['>2km'] = data_chip.apply(lambda row: condition(row), axis=1)

In [95]:
date_val = pd.to_datetime(data_chip['get_time']).dt.date

In [96]:
data_chip = data_chip.assign(date_log = date_val.values)

In [97]:
data_chip.columns

Index(['Imei', 'pawnid', 'open_time', 'event_log', 'longitude', 'latitude',
       'get_time', 'shop_name', 'province', 'customerid', 'codeno', 'fromdate',
       'long_lat', 'latitude_cus', 'longitude_cus', 'long_rad', 'lat_rad',
       'long_rad_cs', 'lat_rad_cs', 'distance_km', 'date_ym_log', 'time_log',
       '>2km', 'date_log'],
      dtype='object')

In [98]:
data_test = data_chip[data_chip['Imei'] == 'A009858']

In [99]:
data_test.to_excel(r'D:\A009858.xlsx')

In [100]:
# data_A001124_0906 = data_chip[data_chip['Imei'] == 'A001124']

In [101]:
# data_A001124_0906.to_excel(r'D:\data_A001124_0906.xlsx')

In [102]:
data_chip2 = data_chip[['Imei', 'pawnid', 'open_time', 'event_log', 'province',
                        'long_lat', 'date_log','time_log']]

In [103]:
#duplicate_row = data_chip2[data_chip2.duplicated(keep=False)]
#duplicate_row.to_excel(r'C:\Users\Admin\Desktop\Project_chip\Data_ tính rule\duplicate_0206.xlsx')

##### Nhóm tín hiệu chip theo ngày - giờ log. Lấy theo đơn vị giờ trong ngày/ 1 ngày = 24 lần

In [104]:
# Chỉ lấy giờ log của con chip
data_chip['date_hour_group'] = data_chip['date_ym_log'] + "-" + data_chip['time_log'].apply(lambda row: row[:2])
data_chip = data_chip.sort_values(by=['pawnid', 'date_log','time_log'], ascending = True)
data_chip['row_id'] = data_chip.groupby(['pawnid','date_hour_group']).cumcount()+1

# Trong 1 giờ chỉ lấy 1 lần log định vị chip
data_chip2 = data_chip[data_chip['row_id'] == 1]

In [105]:
warning_data = data_chip2[['pawnid','codeno','Imei','>2km','long_lat','date_log','time_log','fromdate','event_log','province','shop_name']]

In [106]:
# Chuyển sang dạng bảng kèm điều kiện
df4 = pd.pivot_table(warning_data, values='>2km', index = ['Imei','codeno','pawnid','fromdate','province','shop_name'], columns = 'date_log',aggfunc='sum')

In [107]:
df4.fillna(0, inplace=True)

In [108]:
start_date = pd.to_datetime('2021-05-01').date()

In [109]:
df_result = df4.loc[:,start_date:]

In [110]:
df_result.reset_index(level=['Imei','codeno','pawnid','fromdate','province','shop_name'], inplace=True)

In [111]:
write_data(sheet,'data_10June_2km','data_10June_2km',df_result)

Đã ghi dữ liệu lên sheet data_10June_2km
